In [ ]:

    def move_forward(self):
        if self.direction == "right":
            self.position = (self.position[0], self.position[1] + 1)
        elif self.direction == "left":5cttt
            self.position = (self.position[0], self.position[1] - 1)
        elif self.direction == "up":
            self.position = (self.position[0] - 1, self.position[1])
        elif self.direction == "down":
            self.position = (self.position[0] + 1, self.position[1])

    def turn_left(self):
        if self.direction == "right":
            self.direction = "up"
        elif self.direction == "up":
            self.direction = "left"
        elif self.direction == "left":
            self.direction = "down"
        elif self.direction == "down":
            self.direction = "right"

    def turn_right(self):
        if self.direction == "right":
            self.direction = "down"
        elif self.direction == "down":
            self.direction = "left"
        elif self.direction == "left":
            self.direction = "up"
        elif self.direction == "up":
            self.direction = "right"

    def clean(self):
        print(f"Cleaning position {self.position}")

    def run(self, grid, cleaning_time):
        for _ in range(cleaning_time):
            if grid[self.position[0]][self.position[1]] == "dirty":
                self.clean()
                grid[self.position[0]][self.position[1]] = "clean"

            if self.direction == "right":
                if self.position[1] < len(grid[0]) - 1:
                    self.move_forward()
                else:
                    self.turn_left()
            elif self.direction == "left":
                if self.position[1] > 0:
                    self.move_forward()
                else:
                    self.turn_left()
            elif self.direction == "up":
                if self.position[0] > 0:
                    self.move_forward()
                else:
                    self.turn_left()
            elif self.direction == "down":
                if self.position[0] < len(grid) - 1:
                    self.move_forward()
                else:
                    self.turn_left()


# Example usage:
if __name__ == "__main__":
    # Define the grid (0: clean, 1: dirty)
    grid = [["clean", "dirty", "clean"],
            ["clean", "clean", "dirty"],
            ["dirty", "clean", "clean"]]

    cleaning_time = 30  # Number of time steps for cleaning

    vacuum = VacuumCleaner()
    vacuum.run(grid, cleaning_time)

Cleaning position (0, 1)
Cleaning position (2, 0)
Cleaning position (1, 2)


In [ ]:
from typing import NamedTuple, List

# State definition
class State(NamedTuple):
    monkey_pos: str
    monkey_status: str
    chair_pos: str
    has_stick: bool
    bananas_got: bool

# Initial state
initial_state = State(monkey_pos="atDoor", monkey_status="onFloor", chair_pos="atCenter", has_stick=False, bananas_got=False)

def get_possible_actions(state: State) -> List[str]:
    actions = []

    if state.monkey_status == "onFloor":
        actions.append("move")
        if state.monkey_pos == state.chair_pos:
            actions.append("climb_up")
            actions.append("push")
    else:  # onChair
        if state.has_stick:
            actions.append("wave")
        else:
            actions.append("grab")
        actions.append("climb_down")

    return actions

def apply_action(state: State, action: str) -> State:
    if action == "move":
        # Let's simplify by saying the monkey always moves to the center if not there; else moves to the door.
        new_pos = "atCenter" if state.monkey_pos != "atCenter" else "atDoor"
        return state._replace(monkey_pos=new_pos)

    if action == "climb_up":
        return state._replace(monkey_status="onChair")

    if action == "climb_down":
        return state._replace(monkey_status="onFloor")

    if action == "push":
        # Push chair to under the bananas
        return state._replace(chair_pos="atBananas")

    if action == "grab":
        return state._replace(has_stick=True)

    if action == "wave":
       return state._replace(bananas_got=True)


# Depth-limited search function
def monkey_banana_problem(state: State, path: List[State], depth=0, max_depth=20) -> List[State]:
    if depth > max_depth:
        return []

    if state.bananas_got:
        return path + [state]

    for action in get_possible_actions(state):
        new_state = apply_action(state, action)
        if new_state not in path:  # Avoid loops
            result = monkey_banana_problem(new_state, path + [state], depth+1)
            if result:
                return result
    return []

# Solve the problem
solution = monkey_banana_problem(initial_state, [])
solution


[State(monkey_pos='atDoor', monkey_status='onFloor', chair_pos='atCenter', has_stick=False, bananas_got=False),
 State(monkey_pos='atCenter', monkey_status='onFloor', chair_pos='atCenter', has_stick=False, bananas_got=False),
 State(monkey_pos='atCenter', monkey_status='onChair', chair_pos='atCenter', has_stick=False, bananas_got=False),
 State(monkey_pos='atCenter', monkey_status='onChair', chair_pos='atCenter', has_stick=True, bananas_got=False),
 State(monkey_pos='atCenter', monkey_status='onChair', chair_pos='atCenter', has_stick=True, bananas_got=True)]

In [ ]:
class TicTacToe:
    def __init__(self):
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        self.current_player = 'X'

    def display(self):
        for row in self.board:
            print(' | '.join(row))
            print('-' * 5)

    def is_win(self, player):
        for i in range(3):
            if all([cell == player for cell in self.board[i]]):
                return True
            if all([self.board[j][i] == player for j in range(3)]):
                return True
        if all([self.board[i][i] == player for i in range(3)]) or all([self.board[i][2 - i] == player for i in range(3)]):
            return True
        return False

    def is_draw(self):
        return all(cell != ' ' for row in self.board for cell in row)

class TicTacToeAI(TicTacToe):
    def __init__(self):
        super().__init__()
        self.human_player = 'X'
        self.ai_player = 'O'

    def minimax(self, is_maximizing, alpha, beta):
        if self.is_win(self.ai_player):
            return 1
        if self.is_win(self.human_player):
            return -1
        if self.is_draw():
            return 0

        if is_maximizing:
            max_eval = float('-inf')
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == ' ':
                        self.board[i][j] = self.ai_player
                        eval_val = self.minimax(False, alpha, beta)
                        max_eval = max(max_eval, eval_val)
                        alpha = max(alpha, eval_val)
                        self.board[i][j] = ' '
                        if beta <= alpha:
                            break
            return max_eval
        else:
            min_eval = float('inf')
            for i in range(3):
                for j in range(3):
                    if self.board[i][j] == ' ':
                        self.board[i][j] = self.human_player
                        eval_val = self.minimax(True, alpha, beta)
                        min_eval = min(min_eval, eval_val)
                        beta = min(beta, eval_val)
                        self.board[i][j] = ' '
                        if beta <= alpha:
                            break
            return min_eval

    def find_best_move(self):
        best_score = float('-inf')
        best_move = (-1, -1)

        for i in range(3):
            for j in range(3):
                if self.board[i][j] == ' ':
                    self.board[i][j] = self.ai_player
                    score = self.minimax(False, float('-inf'), float('inf'))
                    self.board[i][j] = ' '
                    if score > best_score:
                        best_score = score
                        best_move = (i, j)
        return best_move

class TicTacToeInteractive(TicTacToeAI):
    def play(self):
        while True:
            if self.is_draw():
                self.display()
                print("It's a draw!")
                break
            if self.current_player == self.human_player:
                self.display()
                row, col = map(int, input("Enter your move (row and column) separated by space (0-2 for both): ").split())
                if self.board[row][col] == ' ':
                    self.board[row][col] = self.human_player
                    if self.is_win(self.human_player):
                        self.display()
                        print("Congratulations! You win!")
                        break
                    self.current_player = self.ai_player
                else:
                    print("Invalid move! Cell already occupied. Try again.")
            else:
                row, col = self.find_best_move()
                self.board[row][col] = self.ai_player
                if self.is_win(self.ai_player):
                    self.display()
                    print("AI wins!")
                    break
                self.current_player = self.human_player

# To play the game, simply run:
game = TicTacToeInteractive()
game.play()


  |   |  
-----
  |   |  
-----
  |   |  
-----
Enter your move (row and column) separated by space (0-2 for both): 0 1
O | X |  
-----
  |   |  
-----
  |   |  
-----
Enter your move (row and column) separated by space (0-2 for both): 1 0
O | X |  
-----
X | O |  
-----
  |   |  
-----
Enter your move (row and column) separated by space (0-2 for both): 2 2
O | X | O
-----
X | O |  
-----
  |   | X
-----
Enter your move (row and column) separated by space (0-2 for both): 0 2 
Invalid move! Cell already occupied. Try again.
O | X | O
-----
X | O |  
-----
  |   | X
-----
Enter your move (row and column) separated by space (0-2 for both): 2 0
O | X | O
-----
X | O |  
-----
X | O | X
-----
Enter your move (row and column) separated by space (0-2 for both): 1 2
O | X | O
-----
X | O | X
-----
X | O | X
-----
It's a draw!


In [ ]:
from queue import PriorityQueue

class Puzzle:
    goal_state = [1, 2, 3, 4, 5, 6, 7, 8, 0]
    heuristic = None
    evaluation_function = None
    needs_hueristic = False
    num_of_instances = 0

    def __init__(self, state, parent, action, path_cost, needs_hueristic=False):
        self.parent = parent
        self.state = state
        self.action = action

        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost

        if needs_hueristic:
            self.needs_hueristic = True
            self.generate_heuristic()
            self.evaluation_function = self.heuristic + self.path_cost
        Puzzle.num_of_instances += 1

    def generate_heuristic(self):
        self.heuristic = 0
        for num in range(9):  # Adjusted range to include 0
            if num == 0:  # Skip the blank tile in heuristic calculation
                continue
            distance = abs(self.state.index(num) - self.goal_state.index(num))
            i = int(distance / 3)
            j = int(distance % 3)
            self.heuristic = self.heuristic + i + j

    def goal_test(self):
        if self.state == self.goal_state:
            return True
        return False

    def generate_child(self):
        children = []
        x = self.state.index(0)
        i = int(x / 3)
        j = int(x % 3)
        legal_actions = self.find_legal_actions(i, j)

        for action in legal_actions:
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x - 3] = new_state[x - 3], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x + 3] = new_state[x + 3], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x - 1] = new_state[x - 1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x + 1] = new_state[x + 1], new_state[x]
            children.append(Puzzle(new_state, self, action, 1, self.needs_hueristic))
        return children

    def find_solution(self):
        solution = []
        solution.append(self.action)
        path = self
        while path.parent != None:
            path = path.parent
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution

    @staticmethod
    def find_legal_actions(i, j):
        legal_action = ['U', 'D', 'L', 'R']
        if i == 0:
            legal_action.remove('U')
        elif i == 2:
            legal_action.remove('D')
        if j == 0:
            legal_action.remove('L')
        elif j == 2:
            legal_action.remove('R')
        return legal_action

def A_star_search(initial_state):
    count = 0
    explored = []
    start_node = Puzzle(initial_state, None, None, 0, True)
    q = PriorityQueue()
    q.put((start_node.evaluation_function, count, start_node))

    while not q.empty():
        node = q.get()[2]
        explored.append(node.state)
        if node.goal_test():
            return node.find_solution()
        children = node.generate_child()
        for child in children:
            if child.state not in explored:
                count += 1
                q.put((child.evaluation_function, count, child))
    return

def main():
    initial_state = []
    valid_input = False

    print("Enter the initial state of the puzzle (row by row) using numbers 0-8:")

    while not valid_input:
        initial_state = []
        try:
            for _ in range(3):
                row = list(map(int, input().split()))
                if len(row) != 3:
                    raise ValueError("Each row should contain exactly 3 numbers.")
                initial_state.extend(row)

            if sorted(initial_state) != list(range(9)):
                raise ValueError("Invalid input. Please enter numbers from 0 to 8, each exactly once.")
            valid_input = True
        except ValueError as e:
            print(str(e))
            print("Try again.")

    result = A_star_search(initial_state)
    if result:
        print("Solution found:")
        for move in result:
            print(move)
    else:
        print("No solution found.")

if __name__ == '__main__':
    main()


Enter the initial state of the puzzle (row by row) using numbers 0-8:


In [ ]:
from typing import Dict, List, Tuple
from collections import deque
from heapq import heappush, heappop

# Define the graph
graph_unweighted = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'F'],
    'F': ['C', 'E']
}

graph_weighted = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'F': 3},
    'D': {'B': 2},
    'E': {'B': 5, 'F': 2},
    'F': {'C': 3, 'E': 2}
}

# Shortest path using BFS
def bfs_shortest_path(graph: Dict[str, List[str]], start: str, end: str) -> List[str]:
    visited = set()
    queue = deque([[start]])

    while queue:
        path = queue.popleft()
        node = path[-1]

        if node == end:
            return path

        for neighbor in graph[node]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(path + [neighbor])

# Shortest path using Lowest-cost-first search
def lowest_cost_first_search(graph: Dict[str, Dict[str, int]], start: str, end: str) -> Tuple[int, List[str]]:
    visited = set()
    priority_queue = [(0, [start])]

    while priority_queue:
        cost, path = heappop(priority_queue)
        node = path[-1]

        if node == end:
            return cost, path

        for neighbor, weight in graph[node].items():
            if neighbor not in visited:
                visited.add(neighbor)
                total_cost = cost + weight
                heappush(priority_queue, (total_cost, path + [neighbor]))

# Test the functions
bfs_path = bfs_shortest_path(graph_unweighted, 'A', 'F')
lowest_cost_path = lowest_cost_first_search(graph_weighted, 'A', 'F')

bfs_path, lowest_cost_path


(['A', 'C', 'F'], (7, ['A', 'C', 'F']))